In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv("CKD.csv")
dataset=pd.get_dummies(dataset,drop_first=True)

In [3]:
independent=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv','wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 
                     'rbc_normal', 'pc_normal','pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes','appet_yes', 'pe_yes', 'ane_yes']]
dependent=dataset[['classification_yes']]

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [8]:
#standardisation
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [21]:
from sklearn.tree import DecisionTreeClassifier

In [33]:
from sklearn.model_selection import GridSearchCV
param_grid={'criterion':['gini','entropy'],'max_features':['log2','sqrt',None],'splitter':['best','random']}
grid= GridSearchCV(DecisionTreeClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['log2', 'sqrt', None],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [35]:
re=grid.cv_results_
grid_predictions=grid.predict(x_test)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predictions)

In [37]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print ("The f1-macro value for best parameter {}:". format(grid.best_params_),f1_macro)
print ("The confusion matrix:\n",cm)
print ("The report:\n",clf_report)

The f1-macro value for best parameter {'criterion': 'gini', 'max_features': 'log2', 'splitter': 'random'}: 0.9503788982803847
The confusion matrix:
 [[44  1]
 [ 5 70]]
The report:
               precision    recall  f1-score   support

       False       0.90      0.98      0.94        45
        True       0.99      0.93      0.96        75

    accuracy                           0.95       120
   macro avg       0.94      0.96      0.95       120
weighted avg       0.95      0.95      0.95       120



In [39]:
#roc-auc
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.007252,0.002815,0.044181,0.012838,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.911692,0.963889,0.947015,1.000000,0.963636,0.957246,0.028610,6
1,0.013245,0.002716,0.075578,0.003441,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.964286,0.946153,0.964572,1.000000,0.981894,0.971381,0.018236,1
2,0.020327,0.005761,0.056566,0.015606,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",1.000000,0.982051,0.929144,0.928571,0.963636,0.960681,0.028415,3
3,0.009713,0.002515,0.030214,0.001731,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.982221,0.910254,0.893717,0.964286,0.981728,0.946441,0.037238,8
4,0.010264,0.001705,0.025476,0.004437,gini,None,best,"{'criterion': 'gini', 'max_features': None, 's...",0.929144,0.909115,0.894531,0.946153,0.981728,0.932134,0.030361,11
5,0.004873,0.004226,0.025978,0.006023,gini,None,random,"{'criterion': 'gini', 'max_features': None, 's...",0.982051,0.964286,0.929513,0.946663,0.963912,0.957285,0.017835,5
6,0.010217,0.001624,0.021134,0.002121,entropy,log2,best,"{'criterion': 'entropy', 'max_features': 'log2...",0.946663,0.892857,0.946663,0.982221,0.945682,0.942817,0.028588,9
7,0.004973,0.003720,0.023961,0.004555,entropy,log2,random,"{'criterion': 'entropy', 'max_features': 'log2...",0.982051,0.964286,0.912029,0.964286,0.963912,0.957313,0.023678,4
8,0.008549,0.002320,0.025916,0.002553,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.964572,0.927778,0.876841,0.963889,0.963247,0.939265,0.034207,10
9,0.004836,0.001452,0.029225,0.001998,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.929513,0.964286,0.982221,0.982221,0.981728,0.967994,0.020435,2


In [41]:
grid.predict([[15,60,3,0,86,15,0.6,138,4,11,33,7700,3.8,0,0,0,0,1,1,0,0,1,1,0,1,0,0]])

array([False])

In [43]:
h=roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])
print(h)

0.9555555555555555


In [45]:
grid.predict([[15,80,0,0,93,17,0.9,136,3.9,16.7,50,6200,5.2,1,0,0,0,1,1,0,0,0,0,0,1,0,0]])

array([False])